In [3]:
import mlflow
from mlflow import MlflowClient

import warnings
warnings.filterwarnings('ignore')

In [4]:
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
EXPERIMENT_NAME = 'portugal-rent-price'


mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [6]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
client

In [7]:
experiment = client.get_experiment_by_name(EXPERIMENT_NAME)
experiment


<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1695056326392, experiment_id='1', last_update_time=1695056326392, lifecycle_stage='active', name='portugal-rent-price', tags={}>

In [9]:
experiment.experiment_id

'1'

In [8]:
query = 'metrics.training_r2_score > 0.4'

In [13]:
beast_model = client.search_runs(
    experiment_ids=experiment.experiment_id,
    max_results=5,
    filter_string=query,
    order_by=['metrics.training_r2_score DESC']
)

In [16]:
run_id = beast_model[0].info.run_id
run_id

'6c34d64fb831426e8e4103f5d350e140'

In [ ]:
# logged_model = f'runs:/{run_id}/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Registro do Modelo 

In [19]:
mlflow.register_model(model_uri=logged_model, name=EXPERIMENT_NAME)

Successfully registered model 'portugal-rent-price'.
2023/09/25 17:40:34 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: portugal-rent-price, version 1
Created version '1' of model 'portugal-rent-price'.


<ModelVersion: aliases=[], creation_timestamp=1695660034926, current_stage='None', description='', last_updated_timestamp=1695660034926, name='portugal-rent-price', run_id='6c34d64fb831426e8e4103f5d350e140', run_link='', source='mlflow-artifacts:/1/6c34d64fb831426e8e4103f5d350e140/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [20]:
from datetime import datetime

In [22]:
client.transition_model_version_stage(EXPERIMENT_NAME, '2', 'Staging', archive_existing_versions=True)

client.update_model_version(EXPERIMENT_NAME, '1', description=f'Atualizado para staging no dia {datetime.now()}')

<ModelVersion: aliases=[], creation_timestamp=1695660034926, current_stage='Staging', description='Atualizado para staging no dia 2023-09-25 17:48:09.427417', last_updated_timestamp=1695660489437, name='portugal-rent-price', run_id='6c34d64fb831426e8e4103f5d350e140', run_link='', source='mlflow-artifacts:/1/6c34d64fb831426e8e4103f5d350e140/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [23]:
client.transition_model_version_stage(EXPERIMENT_NAME, '1', 'Production', archive_existing_versions=True)

client.update_model_version(EXPERIMENT_NAME, '1', description=f'Atualizado para Produção no dia {datetime.now()}')

<ModelVersion: aliases=[], creation_timestamp=1695660034926, current_stage='Production', description='Atualizado para Produção no dia 2023-09-25 17:48:43.929452', last_updated_timestamp=1695660523942, name='portugal-rent-price', run_id='6c34d64fb831426e8e4103f5d350e140', run_link='', source='mlflow-artifacts:/1/6c34d64fb831426e8e4103f5d350e140/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [29]:
client.transition_model_version_stage(EXPERIMENT_NAME, '2', 'Production', archive_existing_versions=True)

client.update_model_version(EXPERIMENT_NAME, '2', description=f'Atualizado para Produção no dia {datetime.now()}')

<ModelVersion: aliases=[], creation_timestamp=1695660779835, current_stage='Production', description='Atualizado para Produção no dia 2023-09-25 17:54:28.639110', last_updated_timestamp=1695660868649, name='portugal-rent-price', run_id='3a4453b0f9274a808b0d7f162453a9da', run_link='', source='mlflow-artifacts:/1/3a4453b0f9274a808b0d7f162453a9da/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>

# Coletando Modelo em produção

In [25]:
last_prod_version = client.get_latest_versions(EXPERIMENT_NAME, stages=['Production'])
last_prod_version

[<ModelVersion: aliases=[], creation_timestamp=1695660034926, current_stage='Production', description='Atualizado para Produção no dia 2023-09-25 17:48:43.929452', last_updated_timestamp=1695660523942, name='portugal-rent-price', run_id='6c34d64fb831426e8e4103f5d350e140', run_link='', source='mlflow-artifacts:/1/6c34d64fb831426e8e4103f5d350e140/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>]

In [28]:
last_prod_version[0].run_id

'6c34d64fb831426e8e4103f5d350e140'